In [1]:
import requests 
from bs4 import BeautifulSoup
import re
from matplotlib import pyplot as plt
import pandas as pd 
import numpy as np
import plotly.express as px
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [265]:
from datetime import date
import plotly.figure_factory as ff

In [2]:
def getHTMLText(url):
    try:
        hd = {"user-agent" : 'Mozilla/5.0'}
        r  = requests.get(url, timeout=30, headers =hd)
        r.raise_for_status()
        r.encoding = r.apparent_encoding
        
      
        return r.text
    except:
        return ""

In [ ]:
https://www.domain.com.au/building-profile/318-russell-street-melbourne-vic-3000?filtertype=onMarket&pagesize=1000&pageno=1

In [13]:
def get_property_rent_history(address):

    raw_html = getHTMLText('https://www.domain.com.au/building-profile/' + address +'?filtertype=rented&pagesize=1000&pageno=1')
    #print(raw_html)
    soup = BeautifulSoup(raw_html, 'html.parser')
    #print(soup)
    list_tags = soup.find_all(class_="css-t7tdkc")
    properties = list_tags[1].find_all(class_ = 'css-3c3rcn')
    df = pd.DataFrame(columns = ['building_name', 'room_no', 'price', 'date' , 'bed', 'bath', 'parking'])
    print(len(properties))
    # loop through each property 
    for pro in properties:
        #print(pro)
        price = pro.find(class_ = 'css-1cq7t6n').text[1:]
        if price[-1] == 'K' or price[-1] == 'k':
            price = float(price[:-1]) * 1000
        else:
            try: 
                price = float(price)
            except ValueError:
                print(price )
                price = None

        #print(price)
        room_no = pro.find('meta')['content'].split('/')[0]
        #print(address1)
        year = pro.find(class_ = 'css-bdklbo').text
        month_date = pro.find(class_ = 'css-rxoubj').text
        #print(year)
        #print(month_date)
        # get room number 
        no_rooms = pro.findAll(class_ = 'css-1ie6g1l')
        res = []
        for r in no_rooms:
            #print(r.find(class_ = 'css-1rzse3v').text)
            temp = r.find(class_ = 'css-1rzse3v').text.split(" ")[0]
            try: 
                res.append(int(temp))
            except ValueError: 
                res.append(0)



        try:
            df = df.append({
                    'building_name' : address,
                    'room_no' : int(room_no), 
                    'price' : price , 
                    'date': pd.to_datetime(year + month_date, format='%Y%b %d'),
                    'bed' : res[0], 
                    'bath': res[1], 
                    'parking' : res[2]
            }, ignore_index=True)
        except ValueError:
            print(room_no)
        
    new_dtypes = {"room_no": pd.Int64Dtype(), 
                  "price": pd.Int64Dtype(),
                    'bed' : pd.Int64Dtype(),
                     'bath' : np.integer,
                    'parking' :pd.Int64Dtype()}
    
    df = df.astype(new_dtypes)
    df['room_type'] = (df['room_no'] %100).astype(str)
    
    df['level'] = df['room_no' ] // 100 
         
    return df 

In [8]:
def med_price_change(df):
    df = df.sort_values("date", ascending = False)
    median_price_before_df = df[df.date <= pd.to_datetime('20200301', format='%Y%m%d')]
    median_price_after_df =  df[df.date > pd.to_datetime('20200301', format='%Y%m%d')]
    median_price_before_df_nona = median_price_before_df.dropna()
    median_price_after_df_nona = median_price_after_df.dropna()
    #median_price_before_df_nona.loc[:, 'room_type'] = median_price_before_df_nona.room_type.astype(str)
    #median_price_after_df_nona.loc[: , 'room_type']= median_price_after_df_nona.room_type.astype(str)
    
    # get med 
    med_before = pd.DataFrame(median_price_before_df_nona.groupby('room_type').price.median())
    med_before = med_before.rename({'price': 'before_price_med'}, axis = 1)

    med_after = pd.DataFrame(median_price_after_df_nona.groupby('room_type').price.median())
    med_after = med_after.rename({'price': 'after_price_med'}, axis = 1)
    room_no = df.groupby('room_type')[['bed', 'bath']].median()
    
    # get difference 
    med_price_df = med_before.join(med_after)
    med_price_df['change_precnetage'] = (med_price_df['after_price_med'] - med_price_df['before_price_med'])/med_price_df['after_price_med'] * 100
    
    # get room info 
    med_price_df = med_price_df.join(room_no)
    
    # plot box 
    df.loc[df.date <= pd.to_datetime('20200301', format='%Y%m%d'),'time_split'] = 'before'

    df.loc[df.date > pd.to_datetime('20200301', format='%Y%m%d'),'time_split'] = 'after'
    df.room_type= df.room_type.astype(str)
    
    fig = px.box(df, x="room_type", y="price", color="time_split",height=400, width=800)
    fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default

    fig.update_xaxes(type='category')
    fig.show()

    
    # plot scatter 
    
    fig = make_subplots(rows=1, cols=2, start_cell="bottom-left",
                       subplot_titles=("Before bedroom =2 ", "After bedroom = 2"))

    rt = 2

    tempdf = median_price_before_df_nona[median_price_before_df.bed == rt]
    fig.add_trace(go.Scatter(x= tempdf.level, y=tempdf.price, 
                            mode='markers',
                            marker=dict(size= list(tempdf['price']//60),
                                        color=[px.colors.qualitative.Dark24[int(r)] for r in tempdf.room_type])),
                          row=1, col=1)


    tempdf = median_price_after_df_nona[median_price_after_df_nona.bed == rt]
    fig.add_trace(go.Scatter(x= tempdf.level, y=tempdf.price, 
                            mode='markers',
                            marker=dict(size= list(tempdf['price'] //60),
                                        color= [px.colors.qualitative.Dark24[int(r)] for r in tempdf.room_type])),
                          row=1, col=2)

    fig.update_layout(height=330, width=1000, title_text="Before and After the pendanmic")

    fig.show()
                                       
                                       
    # plot with time 
#     fig = px.scatter(df[df.bed == rt], x="date", y="price", color="room_type",
#                      size=((df[df.bed == rt])['bed'].astype(int))/50 ,hover_data=['room_type'],width=700, height=400)
    fig = px.scatter(df[df.bed == rt], x="date", y="price", color="room_type",hover_data=['room_type'],width=700, height=400)


    fig.show()

    
    
    
    return med_price_df


### 318 Russell street 

In [15]:
address = '318-russell-street-melbourne-vic-3000'
df_318_rus = get_property_rent_history(address)
med_price_change(df_318_rus)

248


,before_price_med,after_price_med,change_precnetage,bed,bath
room_type,,,,,
1,750,625,-20.000000,2,2
10,730,620,-17.741935,2,2
11,750,<NA>,NaN,2,2
2,775,720,-7.638889,2,2
3,660,580,-13.793103,2,2
4,635,460,-38.043478,2,1
5,545,430,-26.744186,1,1
6,510,420,-21.428571,1,1
7,625,475,-31.578947,2,1


In [6]:
df_318_rus

,building_name,room_no,price,date,bed,bath,parking,room_type,level
0,318-russell-street-melbourne-vic-3000,2606,380,2021-03-03,1,1,0,6,26
1,318-russell-street-melbourne-vic-3000,4305,420,2021-03-03,1,1,0,5,43
2,318-russell-street-melbourne-vic-3000,3504,420,2021-03-02,2,1,0,4,35
3,318-russell-street-melbourne-vic-3000,2305,330,2021-02-24,1,1,0,5,23
4,318-russell-street-melbourne-vic-3000,4207,520,2021-02-24,2,2,0,7,42
...,...,...,...,...,...,...,...,...,...
243,318-russell-street-melbourne-vic-3000,3002,580,2014-12-31,2,2,0,2,30
244,318-russell-street-melbourne-vic-3000,2010,630,2014-12-31,2,2,0,10,20
245,318-russell-street-melbourne-vic-3000,2509,650,2014-12-25,2,2,1,9,25
246,318-russell-street-melbourne-vic-3000,3308,675,2014-12-17,2,2,1,8,33


## New Draft 2021

In [ ]:
df_318_rus = get_property_rent_history(address)

In [297]:

fig = px.scatter(df_318_rus, x="date", y="price", color="room_type", trendline="lowess")
fig.show()

In [304]:
df_318_rus.loc[:, "room_type_2"] = df_318_rus['bed'].astype(int).astype(str) + "b" + df_318_rus['bath'].astype(int).astype(str) + "b"

In [308]:
fig = px.scatter(df_318_rus[df_318_rus.room_type_2.notna()], x="date", y="price", color="room_type_2", width = 700, title= "Building Rented History")
fig.show()

## percentage of dropping Median 

In [ ]:
def perc_change_in_median(df)
    covid_date = '20200301'
    median_price_before_df = df[df.date <= pd.to_datetime(covid_date, format='%Y%m%d')]
    median_price_after_df =  df[df.date > pd.to_datetime(covid_date, format='%Y%m%d')]
    median_price_before_df_nona = median_price_before_df.dropna()
    median_price_after_df_nona = median_price_after_df.dropna()
    #median_price_before_df_nona.loc[:, 'room_type'] = median_price_before_df_nona.room_type.astype(str)
    #median_price_after_df_nona.loc[: , 'room_type']= median_price_after_df_nona.room_type.astype(str)

    # get med 
    med_before = pd.DataFrame(median_price_before_df_nona.groupby('room_type').price.median())
    med_before = med_before.rename({'price': 'before_price_med'}, axis = 1)

    med_after = pd.DataFrame(median_price_after_df_nona.groupby('room_type').price.median())
    med_after = med_after.rename({'price': 'after_price_med'}, axis = 1)
    room_no = df.groupby('room_type')[['bed', 'bath']].median()

    # get difference 
    med_price_df = med_before.join(med_after)
    med_price_df['change_precnetage'] = (med_price_df['after_price_med'] - med_price_df['before_price_med'])/med_price_df['after_price_med'] * 100

    # get room info 
    med_price_df = med_price_df.join(room_no)

    # plot box 
    df.loc[df.date <= pd.to_datetime('20200301', format='%Y%m%d'),'time_split'] = 'before'

    df.loc[df.date > pd.to_datetime('20200301', format='%Y%m%d'),'time_split'] = 'after'
    df.room_type= df.room_type.astype(str)

In [310]:
covid_date = '20200301'
df_318_rus.loc[df_318_rus.date <= pd.to_datetime(covid_date, format='%Y%m%d'),'time_split'] = 'before'
df_318_rus.loc[df_318_rus.date > pd.to_datetime(covid_date, format='%Y%m%d'),'time_split'] = 'after'

In [313]:
median_price_before_df = df_318_rus[df_318_rus.date <= pd.to_datetime(covid_date, format='%Y%m%d')]
median_price_after_df =  df_318_rus[df_318_rus.date > pd.to_datetime(covid_date, format='%Y%m%d')]
perc_change_df = pd.DataFrame(median_price_after_df.groupby("room_type_2").price.median())

In [322]:
perc_change_df = perc_change_df.rename({"price" : 'med_price_after'}, axis = 1)


In [324]:
perc_change_df.loc[:, "med_price_before"] = median_price_before_df.groupby("room_type_2").price.median()
perc_change_df.loc[:, "raw_decrease"] =  perc_change_df["med_price_before"] - perc_change_df["med_price_after"]
perc_change_df.loc[:, "perc_decrease"] = perc_change_df["raw_decrease"]/ perc_change_df["med_price_before"]


In [339]:
perc_change_df

,med_price_after,med_price_before,raw_decrease,perc_decrease
room_type_2,,,,
1b1b,425,521,96,0.184261
2b1b,460,630,170,0.269841
2b2b,600,730,130,0.178082
3b2b,950,1200,250,0.208333


In [351]:
perc_change_df.join(pd.DataFrame(df_318_rus["room_type_2"].value_counts()).rename({"room_type_2": "count"}, axis = 1))

,med_price_after,med_price_before,raw_decrease,perc_decrease,count
room_type_2,,,,,
1b1b,425,521,96,0.184261,69
2b1b,460,630,170,0.269841,58
2b2b,600,730,130,0.178082,112
3b2b,950,1200,250,0.208333,8


In [415]:
table_data = [['room<br>type', 'on market<br>count', 'median<br>price(pw)', 'lowest<br>price', "highest<br>price", "standard<br>deviation"]] 

def fill_table_data(df):
    table_data.append([df.iloc[0,:]["room_type"] , 
                       len(df),"$" + str(df.price.median()),
                       "$" + str(df.price.min()),
                       "$" + str( df.price.max()),
                       "$" + str(round(df.price.std() , 2 ) ) 
                    ])
    
on_market_df.groupby("room_type").apply(fill_table_data)

fig = ff.create_table(table_data, height_constant=60)
fig.layout.width=500
fig0.show()
fig.show()

In [383]:
perc_change_df

,med_price_after,med_price_before,raw_decrease,perc_decrease
room_type_2,,,,
1b1b,425,521,96,0.184261
2b1b,460,630,170,0.269841
2b2b,600,730,130,0.178082
3b2b,950,1200,250,0.208333


In [416]:
tdata = [['room<br>type',"median<br>price<br>before","median<br>price<br>after", "raw<br>difference", "percentage<br>change"]] 


for room_type in perc_change_df.index:
    tdata.append([room_type, 
                       "$" + str(perc_change_df.loc[room_type, 'med_price_before']),
                       "$" + str(perc_change_df.loc[room_type, 'med_price_after']),
                       "$" + str(perc_change_df.loc[room_type, 'raw_decrease']),
                         str(round(perc_change_df.loc[room_type, 'perc_decrease']* 100,2)) + " %",
                    ])
    


colorscale = [[0, '#ffffff'],[.5, '#ffffff'],[1, '#ededed']]

font=['#808080', '#808080', '#808080']
fig = ff.create_table(tdata, height_constant=60, font_colors=font, colorscale = colorscale )
fig.layout.width=500
fig.show()

In [404]:
perc_change_df2

,room_type_2,med_price_after,med_price_before,raw_decrease,perc_decrease,color
0,1b1b,425,521,96,0.184261,NaN
1,2b1b,460,630,170,0.269841,NaN
2,2b2b,600,730,130,0.178082,NaN
3,3b2b,950,1200,250,0.208333,NaN


In [408]:
text_color

[['black', 'black', 'black', 'black'],
 ['black', 'black', 'black', 'black'],
 ['black', 'black', 'black', 'black'],
 ['black', 'black', 'black', 'black'],
 ['black', 'black', 'black', 'black']]

In [406]:
perc_change_df['raw_decrease'].map(lambda p: "red" if p>0 else "green")


room_type_2
1b1b    red
2b1b    red
2b2b    red
3b2b    red
Name: raw_decrease, dtype: object

In [435]:


perc_change_df2 = perc_change_df.reset_index()

perc_change_df2["color"] = perc_change_df2['raw_decrease'].map(lambda p: "red" if p>0 else "green")


col_to_show = ["room_type_2",'med_price_after', 'med_price_before', 'raw_decrease', 'perc_decrease']

text_color = []
n = len(perc_change_df2)
for col in col_to_show:
    if col!='raw_decrease' and col!= "perc_decrease":
        text_color.append(['#808080'] * n)
    else:
        text_color.append(perc_change_df2["color"].to_list())

    
header_text = ['<b>room<br>type</b>',"<b>median<br>rent<br>before</b>","<b>median<br>rent<br>after</b>", "<b>raw<br>difference</b>", "<b>percentage<br>change</b>"]
data=[go.Table(
#     columnwidth = [15,20,30],
    header=dict(values= header_text,
                fill_color='#e3e3e3',
                line_color='white',
                align='center',
                font=dict(color="#808080", family="Lato", size=15),
                height=30
                ),
    cells=dict(values=pd.DataFrame(tdata[1:]).values.T,

               #line_color='darkslategray',
               align='left',
               font=dict(color=text_color, family="Lato", size=15),
               fill_color = ['#f5f5f5'],
               height=30
               ))
]

fig = go.Figure(data=data)
fig.update_layout(width=600, height=600)
fig.show()

In [417]:
pd.DataFrame(tdata[1:]).values.T

array([['1b1b', '2b1b', '2b2b', '3b2b'],
       ['$521', '$630', '$730', '$1200'],
       ['$425', '$460', '$600', '$950'],
       ['$96', '$170', '$130', '$250'],
       ['18.43 %', '26.98 %', '17.81 %', '20.83 %']], dtype=object)

### KNN to identify room types

In [31]:
df = df_318_rus[(df_318_rus.bed==2) &(df_318_rus.bath==2)  & (df_318_rus.parking==0)]

In [179]:
notypes =2 
df.groupby([ "time_split","room_type"]).price.agg([len, min,max])

len  min  max
time_split room_type               
after      1            6  490  660
           10           3  580  700
           2            5  650  800
           3            3  580  600
           4            3  450  650
           7            2  520  670
           8            1  550  550
           9            5  560  680
before     1           10  680  850
           10          19  630  820
           11           2  720  780
           2           12  580  850
           3            7  650  780
           4            2  600  640
           6            1  720  720
           7            2  652  690
           8            9  650  780
           9            9  700  900

In [35]:
df_318_rus[df_318_rus.room_no == 2606]

,building_name,room_no,price,date,bed,bath,parking,room_type,level
0,318-russell-street-melbourne-vic-3000,2606,380,2021-03-03,1,1,0,6,26


In [171]:
df[df.room_type ==10]

,building_name,room_no,price,date,bed,bath,parking,room_type,level,time_split


In [177]:
fig = px.scatter(df[df.room_type =="10"], x="level", y="price", color = "date",width=700, height=400)
fig.show()

In [29]:
fig = px.scatter(df, x="level", y="price", color="room_type",width=700, height=400)

In [30]:
fig.show()

In [34]:
# plot box 
df.loc[df.date <= pd.to_datetime('20200301', format='%Y%m%d'),'time_split'] = 'before'

df.loc[df.date > pd.to_datetime('20200301', format='%Y%m%d'),'time_split'] = 'after'

fig = px.box(df, x="room_type", y="price", color="time_split",height=400, width=800)
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default

fig.update_xaxes(type='category')
fig.show()

#### API Calls

In [83]:
headers = {"accept": "application/json", "X-Api-key" : "key_96722ad209ed03b38756862ecdd76b01"}

In [63]:
# get property ID 
params = {
    "terms" :  "3705/318-russell-street-melbourne-vic-3000", 
    "pageSize" :  20,
    "channel": "Residential"
}
url = 'https://api.domain.com.au/sandbox/v1/properties/_suggest'

In [64]:
r = requests.get(url, params, headers = headers )

In [65]:
res = r.json()

In [66]:
print(res[0]["id"])
print(res[0]["relativeScore"])

CM-7189-NW
100


In [49]:
prop_id = res[0]["id"]

In [54]:
# get size with property id 
params = {
    "id" : prop_id
}
url = 'https://api.domain.com.au/sandbox/v1/properties/' + prop_id

In [61]:
prop_json = requests.get(url, headers = headers ).json()

In [62]:
prop_json['areaSize']

933

In [72]:
# get all size as a new columns 
get_pid_url = 'https://api.domain.com.au/sandbox/v1/properties/_suggest'
pinfo_url =  'https://api.domain.com.au/sandbox/v1/properties/'
# init the column
df_318_rus["areaSize"] = 0
params = {
        "terms" :  "", 
        "pageSize" :  20,
        "channel": "Residential"
        }

# get list history from adv id 
hist_df = pd.DataFrame(columns = [ 'room_no', 'list_id', 'list_date', 'leased_date'])
for i in df_318_rus.index:
    # get property ID 
    
    params["terms"] =  str(df_318_rus.loc[i, "room_no"]) + "/"+ df_318_rus.loc[i, "building_name"]
    res = requests.get(url,params =params, headers = headers ).json()
    prop_id = res[0]["id"]
    prop_json = requests.get(pinfo_url + prop_id, headers = headers ).json()
    df_318_rus.loc[i, "areaSize"] = prop_json["areaSize"]
    
    
    
 
    

KeyError: 0

In [116]:
def get_info_from_list_id(list_id, headers):
    list_url = 'https://api.domain.com.au/sandbox/v1/listings/'
    list_res =  requests.get(list_url + str(list_id), headers = headers )
    if (list_res.status_code != 200):
        return {}
    
    list_json =list_res.json()
    res = {}

    try: 
        list_json['rentalDetails']['leasedDate']
        res['leased_date'] = list_json['rentalDetails']['leasedDate']
    except:
        res['leased_date'] = "n/a"
        
    
    res['list_date'] = list_json['dateListed']
    try:    
        res['price']= list_json['priceDetails']['price']
    except:
        res['price'] = None
        
    res['display_price'] = list_json["priceDetails"]['displayPrice']
    
    res['status'] = list_json['status']
    res['objective'] = list_json['objective']

    return res

In [117]:
# get list history from adv id 
hist_df = pd.DataFrame(columns = [ 'room_no', 'list_id', 'list_date', 'leased_date', 'price', 'display_price','status', 'objective'])
for i in df.index:
    # get property ID 
    params["terms"] =  str(df_318_rus.loc[i, "room_no"]) + "/"+ df_318_rus.loc[i, "building_name"]
    res = requests.get(url,params =params, headers = headers ).json()
    prop_id = res[0]["id"]
    prop_json = requests.get(pinfo_url + prop_id, headers = headers ).json()
    #df_318_rus.loc[i, "areaSize"] = prop_json["areaSize"] 
    current_id = 0 # TODO 
    for photo in prop_json['photos']:
        #print(photo['advertId'])
        if photo['advertId'] != current_id: # TODO 
            list_res = get_info_from_list_id(photo['advertId'], headers)
            list_res['room_no'] = df.loc[i, 'room_no']
            list_res['list_id'] = photo['advertId'] 
            hist_df = hist_df.append(list_res, ignore_index = True )
            # update id 
            current_id = photo['advertId']
            
        

In [129]:
# calculate the up_date , leased_date - list_date 
up_df = hist_df[hist_df.leased_date != 'n/a']
up_df['leased_date'] = pd.to_datetime(up_df['leased_date'], format='%Y-%m-%d')
up_df['list_date'] = up_df['list_date'].astype(str)
up_df['list_date'] = up_df['list_date'].map(lambda p:p.split("T")[0])
up_df['list_date'] = pd.to_datetime(up_df['list_date'], format = "%Y-%m-%d")
up_df['up_date'] = up_df['leased_date'] - up_df['list_date']

<ipython-input-129-6c2db9246440>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-129-6c2db9246440>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-129-6c2db9246440>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-129-6c2

TypeError: unsupported operand type(s) for -: 'DatetimeArray' and 'list'

In [135]:
up_df.loc[:, 'up_date'] = up_df.loc[:,'leased_date'] - up_df.loc[:,'list_date']

In [138]:
up_df = up_df[up_df.up_date.notna()]

In [142]:
up_df['display_price'] = up_df['display_price'].map(lambda p: float(p.split("p")[0][1:]))

<ipython-input-142-395dab26cd3d>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [149]:
up_df

,room_no,list_id,list_date,leased_date,price,display_price,status,objective,up_date
0,4207,14428505,2020-07-31,2020-07-31,NaN,520.0,leased,rent,0 days
1,3809,14717706,2020-11-23,2020-11-23,NaN,560.0,leased,rent,0 days
2,3809,11142287,2017-01-27,2017-02-01,NaN,700.0,leased,rent,5 days
3,1910,14773472,2020-12-15,2020-12-16,NaN,580.0,leased,rent,1 days
4,1910,10664304,2016-06-29,2016-08-04,NaN,720.0,leased,rent,36 days
...,...,...,...,...,...,...,...,...,...
160,4101,10370957,2016-02-16,2016-03-11,NaN,720.0,leased,rent,24 days
162,2108,10370954,2016-02-16,2016-02-16,NaN,650.0,leased,rent,0 days
163,2803,10323451,2016-01-26,2016-02-29,NaN,650.0,leased,rent,34 days
165,2208,10292043,2016-01-12,2016-02-01,NaN,700.0,leased,rent,20 days


In [152]:
up_df.describe()

,price,display_price,up_date
count,20.000000,124.000000,124
mean,756.300000,714.669355,86 days 14:42:34.838709677
std,91.392214,82.607380,172 days 22:53:37.126018822
min,450.000000,450.000000,-305 days +00:00:00
25%,720.000000,668.000000,3 days 00:00:00
50%,766.000000,720.000000,16 days 00:00:00
75%,820.500000,780.000000,34 days 12:00:00
max,850.000000,900.000000,751 days 00:00:00


In [153]:
covid_start ='20200301'
up_df.loc[up_df.leased_date <= pd.to_datetime(covid_start, format='%Y%m%d'),'time_split'] = 'before'

up_df.loc[up_df.leased_date > pd.to_datetime(covid_start, format='%Y%m%d'),'time_split'] = 'after'

/Users/chenbeining/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/chenbeining/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [377]:
up_df["up_days"] = up_df.up_date.map(lambda p: p.days)

In [380]:
(up_df.up_days == 0).sum()

15

In [379]:
fig = px.scatter(up_df, x="up_days", y="display_price",color = 'time_split',hover_data = ['list_id'],width=700, height=400)
fig.show()

In [357]:
fig = px.scatter()

In [358]:
fig.show()

In [125]:
hist_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   room_no        172 non-null    object 
 1   list_id        172 non-null    object 
 2   list_date      170 non-null    object 
 3   leased_date    170 non-null    object 
 4   price          24 non-null     float64
 5   display_price  170 non-null    object 
 6   status         170 non-null    object 
 7   objective      170 non-null    object 
dtypes: float64(1), object(7)
memory usage: 10.9+ KB


In [122]:
(hist_df.leased_date == 'n/a').sum()

46

In [118]:
hist_df

,room_no,list_id,list_date,leased_date,price,display_price,status,objective
0,4207,14428505,2020-07-31T00:07:03Z,2020-07-31,NaN,$520 per week,leased,rent
1,3809,14717706,2020-11-23T05:24:28Z,2020-11-23,NaN,$560 per week,leased,rent
2,3809,11142287,2017-01-27T22:31:21Z,2017-02-01,NaN,$700 per week,leased,rent
3,1910,14773472,2020-12-15T22:49:01Z,2020-12-16,NaN,$580 per week,leased,rent
4,1910,10664304,2016-06-29T05:27:56Z,2016-08-04,NaN,$720 per week,leased,rent
...,...,...,...,...,...,...,...,...
167,3601,9701545,2015-04-11T04:27:09Z,2015-04-17,NaN,$682 per week,leased,rent
168,4804,9449473,2014-12-08T00:18:05Z,n/a,NaN,$600.00 per week,archived,rent
169,3002,9449066,2014-12-07T09:35:05Z,n/a,580.0,$580,archived,rent
170,2010,14463209,2020-08-14T05:17:52Z,n/a,NaN,$400 Per Week,archived,rent


In [107]:
prop_json

{'cadastreType': 'Polygon',
 'onMarketTypes': ['Rent'],
 'status': 'OnMarket',
 'address': '5106/318 Russell Street, Melbourne VIC 3000',
 'addressCoordinate': {'lat': -37.8093, 'lon': 144.96681},
 'addressId': 40688617,
 'areaSize': 933,
 'bathrooms': 2,
 'bedrooms': 2,
 'carSpaces': 0,
 'created': '2021-02-03T03:57:51.331Z',
 'features': ['Study',
  'Spa',
  'Air Conditioning',
  'Heating',
  'Alarm',
  'Pool',
  'Barbeque'],
 'flatNumber': '5106',
 'gnafIds': [{'monthNo': 11, 'yearNo': 2020, 'gnafPID': 'GAVIC425569233'}],
 'id': 'QF-8208-ZZ',
 'isResidential': True,
 'photos': [{'imageType': 'Property',
   'advertId': 14947271,
   'date': '2021-03-02T00:17:14.34Z',
   'fullUrl': 'https://bucket-api.domain.com.au/v1/bucket/image/14947271_1_1_210302_121713-w712-h534',
   'rank': 1},
  {'imageType': 'Property',
   'advertId': 14947271,
   'date': '2021-03-02T00:17:14.34Z',
   'fullUrl': 'https://bucket-api.domain.com.au/v1/bucket/image/14947271_2_1_210302_121713-w712-h534',
   'rank': 

In [110]:
list_url = 'https://api.domain.com.au/sandbox/v1/listings/'
list_json = requests.get(list_url + str(photo['advertId']), headers = headers )

In [115]:
list_json.status_code

200

In [ ]:
df = pd.DataFrame(columns = ['building_name', 'room_no', 'price', 'date' , 'bed', 'bath', 'parking'])

In [74]:
res

{'title': 'Internal Server Error',
 'status': 500,
 'detail': 'The request was canceled due to the configured HttpClient.Timeout of 10 seconds elapsing.',
 'traceId': '5f87f9b7f89d094fb3f1caf560289f74'}

In [77]:
df_318_rus.areaSize.value_counts()

933    205
0       41
937      2
Name: areaSize, dtype: int64

In [78]:
prop_json

{'cadastreType': 'Polygon',
 'onMarketTypes': [],
 'status': 'OffMarket',
 'address': '3306/318 Russell Street, Melbourne VIC 3000',
 'addressCoordinate': {'lat': -37.80935, 'lon': 144.9668},
 'addressId': 40693967,
 'areaSize': 933,
 'bathrooms': 1,
 'bedrooms': 1,
 'carSpaces': 0,
 'created': '2021-02-03T02:10:29.581Z',
 'features': ['Study', 'Air Conditioning', 'Heating', 'Alarm', 'Barbeque'],
 'flatNumber': '3306',
 'gnafIds': [{'monthNo': 11, 'yearNo': 2020, 'gnafPID': 'GAVIC425569033'}],
 'id': 'FZ-6415-LH',
 'isResidential': True,
 'photos': [{'imageType': 'Property',
   'advertId': 11060681,
   'date': '2017-01-07T23:25:37.193Z',
   'fullUrl': 'https://bucket-api.domain.com.au/v1/bucket/image/w800-h529-11060681_1_pi_170106_033333',
   'rank': 1},
  {'imageType': 'Property',
   'advertId': 11060681,
   'date': '2017-01-07T23:25:37.193Z',
   'fullUrl': 'https://bucket-api.domain.com.au/v1/bucket/image/w800-h529-11060681_2_pi_170106_033334',
   'rank': 2},
  {'imageType': 'Propert

### Get Current Listing 

In [217]:
page_no = 1
on_market_list = []
while True: 
    cur_list_url = 'https://www.domain.com.au/building-profile/' + address + '?filtertype=forRent&pagesize=10&pageno=' + str(page_no)
    raw_html = getHTMLText(cur_list_url)
    page_no += 1 

    soup = BeautifulSoup(raw_html, 'html.parser')
    #print(soup)
    list_tags = soup.find_all(class_ = "css-t7tdkc")
    properties = list_tags[0].find_all(class_ =  'css-3c3rcn', attrs = {"data-testid": 'nearby-properties__on-market-property'} )
    for prop in properties: 
        
        list_id = prop.find_all("a")[0]['href'].split("-")[-1]
        #print(list_id)
        on_market_list.append(list_id)
    if(len(properties) == 0):
        break 

    

In [218]:
on_market_list 

['14928673',
 '14958705',
 '14958385',
 '14957690',
 '14955899',
 '14955799',
 '14955716',
 '14947271',
 '14946865',
 '14940057',
 '14936089',
 '14930793',
 '14920068',
 '14919124',
 '14917397',
 '14915101',
 '14914813',
 '14910807',
 '14906069',
 '14906056',
 '14906033',
 '14905866',
 '14905858',
 '14897818',
 '14896727',
 '14883526',
 '14883178',
 '14883039',
 '14879991',
 '14877863',
 '14871156',
 '14869618',
 '14868914',
 '14868220',
 '14868083',
 '14867181',
 '14864460',
 '14855089',
 '14854777',
 '14854774',
 '14853599',
 '14830157',
 '14822799',
 '14811181',
 '14807103',
 '14779144',
 '14757444',
 '14729672',
 '14718152',
 '14662122',
 '14644385',
 '14644357',
 '14637261',
 '14590148',
 '14590023',
 '14587759',
 '14585512',
 '14585274',
 '14571077',
 '14559609',
 '14536377',
 '14521642',
 '14435555',
 '14425646',
 '14424258',
 '14408965',
 '14391282',
 '14318978',
 '14298280',
 '14283215',
 '13952555']

In [241]:
on_market_df = pd.DataFrame(columns = ["list_id","status", 'room_no', 'price','list_date' , 'bed', 'bath', 'parking', 'agency_id'])

In [244]:
# fetch from listing API 
for list_id in on_market_list:
    list_url = 'https://api.domain.com.au/sandbox/v1/listings/'
    list_res =  requests.get(list_url + str(list_id), headers = headers )
    if (list_res.status_code == 200):
        json_res = list_res.json()
        print(list_id)
        try: 
            carspaces = json_res['carspaces'] 
        except: 
            carspaces = 0 
        try:
            room_no =  re.search("[0-9][0-9]*", json_res["addressParts"]["unitNumber"]).group()
        except:
            room_no = re.search("[0-9][0-9]*", json_res["addressParts"]["streetNumber"]).group()
            
        raw_price = json_res['priceDetails']['displayPrice']
        price = float(re.search("[0-9][0-9]*", raw_price).group())
        re.search("[0-9][0-9]*[^0-9]", raw_price)
        on_market_df = on_market_df.append({"list_id" : list_id,
                             'room_no' : room_no,
                             "price" : price,
                             "list_date" : json_res['dateListed'] ,
                              "status" : json_res["status"],
                             "bed" : float(json_res['bedrooms']), 
                             "bath" : float(json_res['bathrooms']),
                             "parking" : float(carspaces ) , 
                             "agency_id": json_res['advertiserIdentifiers']['advertiserId'] 
                            
                            }, ignore_index = True)
        
        
    
   
        
        

14928673
14940057
14936089
14930793
14920068
14919124
14917397
14915101
14914813
14910807
14906069
14906056
14906033
14905866
14905858
14897818
14896727
14883526
14883178
14883039
14879991
14877863
14871156
14869618
14868914
14868220
14868083
14867181
14864460
14855089
14854777
14854774
14853599
14830157
14822799
14811181
14807103
14779144
14757444
14729672
14718152
14662122
14644385
14644357
14637261
14590148
14590023
14587759
14585512


In [250]:
on_market_df.loc[:, "list_date"] = pd.to_datetime(on_market_df['list_date'].map(lambda p: p.split("T")[0]), format = "%Y-%m-%d")


AttributeError: 'Timestamp' object has no attribute 'split'

In [252]:
on_market_df.loc[:, "up_days"] =  on_market_df["list_date"].map(lambda p: (date.today() - p.date()).days)

In [254]:
on_market_df.loc[:, "room_type"] = on_market_df['bed'].astype(int).astype(str) + "b" + on_market_df['bath'].astype(int).astype(str) + "b"

In [255]:
on_market_df
        

,list_id,status,room_no,price,list_date,available_date,bed,bath,parking,agency_id,up_days,room_type
0,14928673,live,3705,380.0,2021-02-23,NaN,1.0,1.0,0.0,18765,11,1b1b
1,14940057,new,4508,550.0,2021-02-26,NaN,2.0,1.0,0.0,11842,8,2b1b
2,14936089,live,3009,600.0,2021-02-25,NaN,2.0,2.0,0.0,11842,9,2b2b
3,14930793,live,3502,600.0,2021-02-23,NaN,2.0,2.0,0.0,11842,11,2b2b
4,14920068,new,4509,650.0,2021-02-18,NaN,2.0,2.0,0.0,11842,16,2b2b
5,14919124,live,3004,480.0,2021-02-18,NaN,2.0,1.0,0.0,4757,16,2b1b
6,14917397,live,1302,550.0,2021-02-18,NaN,2.0,2.0,0.0,232,16,2b2b
7,14915101,live,4506,430.0,2021-02-17,NaN,1.0,1.0,0.0,11842,17,1b1b
8,14914813,live,1009,650.0,2021-02-17,NaN,2.0,2.0,0.0,11842,17,2b2b
9,14910807,live,4710,650.0,2021-02-16,NaN,2.0,2.0,0.0,11842,18,2b2b


In [259]:
on_market_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   list_id         49 non-null     object        
 1   status          49 non-null     object        
 2   room_no         49 non-null     object        
 3   price           49 non-null     float64       
 4   list_date       49 non-null     datetime64[ns]
 5   available_date  0 non-null      float64       
 6   bed             49 non-null     float64       
 7   bath            49 non-null     float64       
 8   parking         49 non-null     float64       
 9   agency_id       49 non-null     object        
 10  up_days         49 non-null     int64         
 11  room_type       49 non-null     object        
dtypes: datetime64[ns](1), float64(5), int64(1), object(5)
memory usage: 4.7+ KB


In [293]:
fig0 = px.strip(on_market_df, x="room_type", y="price", hover_data = ['up_days', 'list_id'], color = "room_type",width = 500, height = 400, title = "71 properties are currently On Market for Rent" )
fig0.update_layout(showlegend=False)
fig0.show()

In [294]:
table_data = [['room<br>type', 'on market<br>count', 'median<br>price(pw)', 'lowest<br>price', "highest<br>price", "standard<br>deviation"]] 

def fill_table_data(df):
    table_data.append([df.iloc[0,:]["room_type"] , 
                       len(df),"$" + str(df.price.median()),
                       "$" + str(df.price.min()),
                       "$" + str( df.price.max()),
                       "$" + str(round(df.price.std() , 2 ) ) 
                    ])
    
on_market_df.groupby("room_type").apply(fill_table_data)

fig = ff.create_table(table_data, height_constant=60)
fig.layout.width=500
fig0.show()
fig.show()

### Get One Prop hist

In [ ]:
listing_id = 14718152


In [440]:
prop_id = 'PG-9328-YD'
pinfo_url = 'https://api.domain.com.au/sandbox/v1/properties/'
prop_json = requests.get(pinfo_url + prop_id, headers = headers ).json()
#df_318_rus.loc[i, "areaSize"] = prop_json["areaSize"] 
current_id = 0 # TODO 
for photo in prop_json['photos']:
    print(photo['advertId'])
    if photo['advertId'] != current_id: # TODO 
        #list_res = get_info_from_list_id(photo['advertId'], headers)
        #list_res['room_no'] = df.loc[i, 'room_no']
        #list_res['list_id'] = photo['advertId'] 
        #hist_df = hist_df.append(list_res, ignore_index = True )
        # update id 
        current_id = photo['advertId']
            

14906056
14906056
14906056
14906056
14906056
14906056
14906056
14906056
14906056
13688573
13688573
13688573
13688573
13688573
13688573
13688573
13688573
10963374
10963374
10963374
10963374
10963374
10963374
10963374
10963374


In [465]:
room_no = 1102
prop_url = 'https://www.domain.com.au/property-profile/' + str(room_no) + "-" + address
raw_html = getHTMLText(prop_url)
soup = BeautifulSoup(raw_html, 'html.parser')
#print(soup)
list_tags = soup.find_all(class_="css-m3i618")
timeline_df = pd.DataFrame(columns = ["Date", "Price","days", "Type", "Agency"])
# for each record
for hist in list_tags[0].find_all(class_ = "css-16ezjtx"):
    temp = hist.find_all(class_ = 'css-zwto9f')
    if len(temp) == 1:
        days = "N/A"
    else:
        days = temp[1].text
        
    
    this_type = hist.find(class_ = 'css-bl2deh')
    if this_type is None:
        this_type = hist.find(class_ = 'css-jcs3kb').text
    else:
        this_type = this_type.text
        
    timeline_df = timeline_df.append({
        "Date" : hist.find(class_ = 'css-vajoca').text + hist.find(class_ = 'css-1qi20sy').text,
        "Price" : hist.find(class_ = 'css-6xjfcu').text + hist.find(class_ = 'css-obiveq').text,
        "Type" : this_type, 
        "days" : days ,
        "Agency" : temp[0].text
        
    },
                      ignore_index= True)
    
    

In [468]:
rent_df = timeline_df[timeline_df.Type == "RENTED"]

In [476]:
rent_df

,Date,Price,days,Type,Agency,date_date,price_num
0,Jan2020,$800PER WEEK,66,RENTED,Melcorp Real Estate,2020-01-01,800.0
1,Oct2017,$770PER WEEK,98,RENTED,Melcorp Property - Melbourne,2017-10-01,770.0


In [510]:
rent_df["description"] = "<b>" + rent_df["price_num"].astype(int).astype(str)  + "</b> per week<br>Listed for <b>" + rent_df["days"] + " </b>days<br>by " + rent_df["Agency"]

In [511]:
rent_df["date_date"] = pd.to_datetime(rent_df["Date"], format = "%b%Y")
rent_df["price_num"] = rent_df["Price"].map(lambda p: float(p.split("P")[0][1:]))

In [512]:
fig = go.Figure(data=go.Scatter(x=rent_df['date_date'],
                                y=rent_df["price_num"],
                                mode = "lines+text+markers",
                               text = rent_df["description"],
                               textposition="top center",
                                textfont = {'family': "Times", 'size': 20},
                                marker=dict(size=15)
                               ),
                               )

fig.update_layout(yaxis_range=[600,900], xaxis_range = ['2017-01-01', '2020-06-08'])
fig.show()

In [447]:
len(list_tags[0].find_all(class_ = "css-16ezjtx"))

3

In [448]:
test = list_tags[0].find_all(class_ = "css-16ezjtx")[0]

In [463]:
print(hist.find(class_ = 'css-vajoca').text)
print(hist.find(class_ = 'css-1qi20sy').text)
print(hist.find(class_ = 'css-bl2deh').text)
print(hist.find(class_ = 'css-6xjfcu').text)
print(hist.find(class_ = 'css-obiveq').text)
print(hist.find_all(class_ = 'css-zwto9f'))
#print(test.find_all(class_ = 'css-1oi8ih3'))

Sep
2011


AttributeError: 'NoneType' object has no attribute 'text'

In [441]:
prop_json

{'cadastreType': 'Polygon',
 'onMarketTypes': ['Rent'],
 'status': 'OnMarket',
 'address': '1102/318 Russell Street, Melbourne VIC 3000',
 'addressCoordinate': {'lat': -37.8093, 'lon': 144.96681},
 'addressId': 40693237,
 'areaSize': 933,
 'bathrooms': 2,
 'bedrooms': 2,
 'carSpaces': 0,
 'created': '2021-02-03T03:47:36.614Z',
 'features': ['Air Conditioning',
  'Alarm',
  'Barbeque',
  'Ensuite',
  'Heating',
  'Pool',
  'Spa'],
 'flatNumber': '1102',
 'gnafIds': [{'monthNo': 11, 'yearNo': 2020, 'gnafPID': 'GAVIC425568761'}],
 'id': 'PG-9328-YD',
 'isResidential': True,
 'photos': [{'imageType': 'Property',
   'advertId': 14906056,
   'date': '2021-03-05T01:05:40.063Z',
   'fullUrl': 'https://bucket-api.domain.com.au/v1/bucket/image/13638038_2_1_191010_124511-w800-h530',
   'rank': 1},
  {'imageType': 'Property',
   'advertId': 14906056,
   'date': '2021-03-05T01:05:40.063Z',
   'fullUrl': 'https://bucket-api.domain.com.au/v1/bucket/image/13638038_1_1_191010_124511-w800-h530',
   'ran

## Agent 

In [330]:
agency_df= pd.DataFrame(on_market_df.agency_id.value_counts())

In [356]:
agency_df

,agency_id
11842,35
8389,2
4757,2
24371,2
24974,1
190,1
18765,1
29243,1
32569,1
232,1


In [355]:

for aid in agency_df.index:
    agency_url =  "https://api.domain.com.au/sandbox/v1/agencies/"
    res =   requests.get(list_url + str(aid), headers = headers )
    
    if res.status_code != 200:
        print(aid)
        continue 
    res_json = res.json()
    agency_df.loc[aid, "agency_name"] = res_json["name"]
    
    

11842
8389
4757
24371
24974
190
18765
29243
32569
232
17347
30082


In [354]:
res_json

{'message': '{"type":"https://tools.ietf.org/html/rfc7231#section-6.5.4","title":"Not Found","status":404,"traceId":"00-d246020383d04c42a83aacd41d495d15-95b222785aa77543-01"}'}

In [338]:
agency_df

,agency_id
11842,35
8389,2
4757,2
24371,2
24974,1
190,1
18765,1
29243,1
32569,1
232,1


### Aurora

In [9]:
(df_aurora.price.isna()).sum()

NameError: name 'df_aurora' is not defined

In [ ]:
address = '228-la-trobe-street-melbourne-vic-3000'
df_aurora = get_property_rent_history(address)
med_price_change(df_aurora)

In [ ]:
med_price_change(df_aurora)

### Zen

# Draft 

In [ ]:
address = '500-elizabeth-street-melbourne-vic-3000'
df = get_property_rent_history(address)
df["year_month"]= df.date.dt.strftime("%Y %b")
df['bed_bath'] = df['bed'].astype(str) + "_" + df['bath'].astype(str)

In [ ]:
df['bed_bath'] = df['bed'].astype(str) + "_" + df['bath'].astype(str)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.room_type.unique()

In [ ]:
plt.figure(figsize = (10,10))
fig = df.groupby(['year_month', 'bed_bath']).price.min().unstack().plot()
plt.axvline(x= 47) 
df.groupby(['year_month', 'bed_bath']).price.max().unstack().plot()
plt.axvline(x= 47) 
df.groupby(['year_month', 'bed_bath']).price.median().unstack().plot()
plt.axvline(x= 47) 

In [ ]:
plt.figure(figsize = (15 ,5 ))
g = sns.scatterplot(
    data=df, x='year_month' , y="price", hue="bed", size="bath",sizes=(30, 150),x_jitter=True, y_jitter=True, alpha=0.8,
    palette=sns.color_palette("Set1", df.bed.nunique())
)
_ = plt.xticks(rotation=90)
_ = plt.legend(loc='upper left')
g.axes.invert_xaxis()

In [ ]:
for i in range(1,4):
    fig = px.scatter(df[df.bed == i], x="date", y="price", color="room_type",
                     size=((df[df.bed == i])['level'].astype(int))/50 ,hover_data=['room_type'],width=700, height=400)

    fig.show()


## Linear Regression 

In [ ]:
df = df.sort_values("date", ascending = False)
median_price_before_df = df[df.date <= pd.to_datetime('20200301', format='%Y%m%d')]
median_price_after_df =  df[df.date > pd.to_datetime('20200301', format='%Y%m%d')]

In [ ]:
median_price_before_df_nona = median_price_before_df.dropna()
median_price_after_df_nona = median_price_after_df.dropna()

In [ ]:
median_price_before_df_nona['room_type'] = median_price_before_df_nona.room_type.astype(str)
median_price_after_df_nona['room_type']= median_price_after_df_nona.room_type.astype(str)

In [ ]:
median_price_after_df_nona.info()

In [ ]:
median_price_after_df_nona.index

### One hot encoding 

In [ ]:
enc = OneHotEncoder()
res_array = enc.fit_transform(median_price_after_df_nona[['room_type']]).toarray()
onehot_res = pd.DataFrame(res_array, columns = enc.get_feature_names(['room_type']), index = median_price_after_df_nona.index)

In [ ]:
median_price_after_df_nona = median_price_after_df_nona.join(onehot_res)

In [ ]:
x = median_price_after_df_nona [['bed', 'level','room_type_1',
       'room_type_10', 'room_type_11', 'room_type_12', 'room_type_2',
       'room_type_3', 'room_type_5', 'room_type_6', 'room_type_7',
       'room_type_8', 'room_type_9']]
y = median_price_after_df_nona['price']

### Run the Model 

In [ ]:
reg_after= LinearRegression().fit(x, y)

In [ ]:
reg_after

In [ ]:
from sklearn.metrics import median_absolute_error

y_pred = reg_after.predict(x)

mae = median_absolute_error(y, y_pred)
fig, ax = plt.subplots(figsize=(5, 5))
plt.scatter(y, y_pred)
ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="--", c="red")
plt.text(3, 20, string_score)
plt.title('Linear regression Accuracy on Train Set')
plt.ylabel('Model predictions')
plt.xlabel('Truths')
# plt.xlim([0, 27])
# _ = plt.ylim([0, 27])

###  Coefficients 

In [ ]:

coefs = pd.DataFrame(
    reg_after.coef_,
    columns=['Coefficients'], index=x.columns
)
coefs.plot(kind='barh', figsize=(9, 7))
plt.title('marginal price change')
plt.axvline(x=0, color='.5')
plt.subplots_adjust(left=.3)

In [ ]:

coefs = pd.DataFrame(
    reg_after.coef_ * x.std(axis=0),
    columns=['Coefficients'], index=x.columns
)

coefs

In [ ]:

coefs.plot(kind='barh', figsize=(9, 7))
plt.title('Feature importance')
plt.axvline(x=0, color='.5')
plt.subplots_adjust(left=.3)

## Change in Median Price

In [ ]:
med_before = pd.DataFrame(median_price_before_df_nona.groupby('room_type').price.median())
med_before = med_before.rename({'price': 'before_price_med'}, axis = 1)

med_after = pd.DataFrame(median_price_after_df_nona.groupby('room_type').price.median())
med_after = med_after.rename({'price': 'after_price_med'}, axis = 1)


In [ ]:
room_no = df.groupby('room_type')[['bed', 'bath']].median()

In [ ]:
med_price_df = med_before.join(med_after)
med_price_df['change_precnetage'] = (med_price_df['after_price_med'] - med_price_df['before_price_med'])/med_price_df['after_price_med'] * 100

In [ ]:
med_price_df = med_price_df.join(room_no)


In [ ]:
med_price_df

### Make Plots 

In [ ]:


fig = make_subplots(rows=3, cols=2, start_cell="bottom-left",
                   subplot_titles=("Before bedroom = 1 ", "After bedroom = 1", "Before bedroom = 2",
                                   "After bedroom = 2",'Before bedroom = 3', 'After bedroom = 3'))

for rt in range(1,4):

    tempdf = median_price_before_df[median_price_before_df.bed == rt]
    fig.add_trace(go.Scatter(x= tempdf.level, y=tempdf.price, 
                            mode='markers',
                            marker=dict(size= list(tempdf['price']//60),
                                        color=[px.colors.qualitative.Dark24[int(r)] for r in tempdf.room_type])),
                          row=rt, col=1)

for rt in range(1,4):
    tempdf = median_price_after_df_nona[median_price_after_df_nona.bed == rt]
    fig.add_trace(go.Scatter(x= tempdf.level, y=tempdf.price, 
                            mode='markers',
                            marker=dict(size= list(tempdf['price'] //60),
                                        color= [px.colors.qualitative.Dark24[int(r)] for r in tempdf.room_type])),
                          row=rt, col=2)

fig.update_layout(height=1000, width=1000, title_text="Before and After the pendanmic")

fig.show()

## Box plot split by room type 

In [ ]:
df.loc[df.date <= pd.to_datetime('20200301', format='%Y%m%d'),'time_split'] = 'before'

df.loc[df.date > pd.to_datetime('20200301', format='%Y%m%d'),'time_split'] = 'after'
df.room_type= df.room_type.astype(str)

In [ ]:
fig = px.box(df, x="room_type", y="price", color="time_split")
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default

fig.update_xaxes(type='category')
fig.show()